# Panle Regression - Firm Characteristics

### Random Effects Panel Regression

In [33]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [34]:
df = pd.read_csv("Dataframes/macro_regression.csv")
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,CBOE Crude Oil ETF Volatility Index,90-Day AA Financial Commercial Paper Interest Rate,Inflation Risk Premium,"University of Michigan: Consumer Sentiment, Index 1966:Q1=100",Unemployment Rate
0,AVY.N,01/01/13,Materials,11.178,11.178,22.538500,0.166000,0.359537,76.666667,7.733333
1,AVY.N,01/04/13,Materials,2.482,2.482,23.468125,0.143750,0.360572,81.666667,7.533333
2,AVY.N,01/07/13,Materials,1.068,1.068,24.099219,0.125625,0.463789,81.566667,7.233333
3,AVY.N,01/10/13,Materials,8.095,8.095,19.740938,0.125806,0.441854,76.933333,6.933333
4,AVY.N,01/01/14,Materials,1.471,1.471,19.563770,0.126230,0.465460,80.933333,6.666667
...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,01/10/21,Retailing,17.194,17.194,43.036308,0.140755,0.382645,69.900000,4.200000
19197,POOL.OQ,01/01/22,Retailing,40.267,40.267,52.695806,0.467234,0.380946,63.133333,3.800000
19198,POOL.OQ,01/04/22,Retailing,34.342,34.342,50.457581,1.394118,0.495588,57.866667,3.600000
19199,POOL.OQ,01/07/22,Retailing,1.503,1.503,49.861562,2.885882,0.423045,56.100000,3.566667


In [35]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

In [36]:
# Calculate the 0.15th and 99.85th percentile
pct_0_15 = np.percentile(df['Earnings Per Share - Actual Surprise'], 0.5)
pct_99_85 = np.percentile(df['Earnings Per Share - Actual Surprise'], 99.5)

# Remove the data points outside the 0.15th and 99.85th percentile
df_clean = df[(df['Earnings Per Share - Actual Surprise'] >= pct_0_15) & (df['Earnings Per Share - Actual Surprise'] <= pct_99_85)].copy()

Standardising

In [37]:
rescale = df_clean
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["CBOE Crude Oil ETF Volatility Index"] = MinMaxScaler().fit_transform(np.array(rescale["CBOE Crude Oil ETF Volatility Index"]).reshape(-1,1))
rescale["90-Day AA Financial Commercial Paper Interest Rate"] = MinMaxScaler().fit_transform(np.array(rescale["90-Day AA Financial Commercial Paper Interest Rate"]).reshape(-1,1))
rescale["Inflation Risk Premium"] = MinMaxScaler().fit_transform(np.array(rescale["Inflation Risk Premium"]).reshape(-1,1))
rescale["University of Michigan: Consumer Sentiment, Index 1966:Q1=100"] = MinMaxScaler().fit_transform(np.array(rescale["University of Michigan: Consumer Sentiment, Index 1966:Q1=100"]).reshape(-1,1))
rescale["Unemployment Rate"] = MinMaxScaler().fit_transform(np.array(rescale["Unemployment Rate"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,CBOE Crude Oil ETF Volatility Index,90-Day AA Financial Commercial Paper Interest Rate,Inflation Risk Premium,"University of Michigan: Consumer Sentiment, Index 1966:Q1=100",Unemployment Rate
0,AVY.N,2013-01-01,Materials,11.178,11.178,0.058434,0.015981,0.552998,0.480156,0.443262
1,AVY.N,2013-01-04,Materials,2.482,2.482,0.069092,0.010593,0.555632,0.596887,0.421986
2,AVY.N,2013-01-07,Materials,1.068,1.068,0.076327,0.006205,0.818318,0.594553,0.390071
3,AVY.N,2013-01-10,Materials,8.095,8.095,0.026361,0.006249,0.762495,0.486381,0.358156
4,AVY.N,2014-01-01,Materials,1.471,1.471,0.024330,0.006351,0.822572,0.579767,0.329787
...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-01-10,Retailing,17.194,17.194,0.293433,0.009868,0.611807,0.322179,0.067376
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.404175,0.088919,0.607483,0.164202,0.024823
19198,POOL.OQ,2022-01-04,Retailing,34.342,34.342,0.378514,0.313346,0.899246,0.041245,0.003546
19199,POOL.OQ,2022-01-07,Retailing,1.503,1.503,0.371681,0.674548,0.714626,0.000000,0.000000


In [38]:
df_panel = df_clean.copy()

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [39]:
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)

Regression with absolute surprise values

In [40]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "CBOE Crude Oil ETF Volatility Index":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,8.578e-08
Estimator:,PanelOLS,R-squared (Between):,-7.481e+21
No. Observations:,19013,R-squared (Within):,-1.645e+20
Date:,"Sat, Mar 04 2023",R-squared (Overall):,-3.761e+21
Time:,13:09:50,Log-likelihood,-8.492e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.0003
Entities:,502,P-value,1.0000
Avg Obs:,37.875,Distribution:,"F(5,18467)"
Min Obs:,2.0000,,
Max Obs:,40.000,F-statistic (robust):,0.0003


By Industry: example tech industry

In [41]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = group.loc[:, "CBOE Crude Oil ETF Volatility Index":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_61524/2986103428.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_61524/2986103428.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0000
Estimator:,PanelOLS,R-squared (Between):,0.6728
No. Observations:,731,R-squared (Within):,-1.3302
Date:,"Sat, Mar 04 2023",R-squared (Overall):,-0.4220
Time:,13:09:50,Log-likelihood,-2781.8
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.0000
Entities:,19,P-value,1.0000
Avg Obs:,38.474,Distribution:,"F(5,668)"
Min Obs:,28.000,,
Max Obs:,40.000,F-statistic (robust):,3.615e-31
